In [9]:
import json

def add_instructions_and_file_sizes(file_suffix):
    instructions = []
    with open('perf_results_{}.txt'.format(file_suffix)) as file:
        for line in file:
            if 'instructions' in line:
                split_line = line.split(' ')
                for ind, part in enumerate(split_line):
                    if part == 'instructions':
                        length = ind
                instructions.append(int(''.join(split_line[:length])))
    file_sizes = {}
    with open('file_sizes_{}.txt'.format(file_suffix)) as file:
        for line in file:
            split_line = line.split()
            file_sizes[split_line[1]] = int(split_line[0])/1000
    with open('workflow_{}.json'.format(file_suffix)) as file:
        json_data = json.load(file)
        for node in json_data['signals']:
            name = node['name']
            if name in file_sizes:
                node['size'] = file_sizes[node['name']]
        print(len(json_data['processes']))
        print(len(instructions))
        for node, num_instr in zip(json_data['processes'], instructions):
            node['config']['executor']['instructions'] = num_instr
    with open('workflow_{}_decorated.json'.format(file_suffix), 'w+') as outfile:
        json.dump(json_data, outfile, indent=2)

In [42]:
add_instructions_and_file_sizes('0.25')
add_instructions_and_file_sizes('1.0')
add_instructions_and_file_sizes('2.0')

In [13]:
def add_requests(mem_request, cpu_request, filename):
    with open(filename) as file:
        json_data = json.load(file)
        for node in json_data['processes']:
            if 'config' in node:
                executor = node['config']['executor']
                executor['memRequest'] = mem_request
                executor['cpuRequest'] = cpu_request
    with open(filename[:-5]+'_decorated.json', 'w+') as outfile:
        json.dump(json_data, outfile, indent=2)

In [19]:
add_requests(1000, 2500, 'workflow_2.0_decorated.json')

In [11]:
add_instructions_and_file_sizes('soykb')

52
50


In [14]:
add_requests(1000, 2500, 'workflow_soykb_decorated.json')

In [16]:
def add_file_parts(filename, parts_by_task_name):
    with open(filename) as file:
        json_data = json.load(file)
        for node in json_data['processes']:
            if 'config' in node and node['name'] in parts_by_task_name:
                executor = node['config']['executor']
                executor['filePart'] = parts_by_task_name[node['name']]
    with open(filename[:-5]+'_decorated.json', 'w+') as outfile:
        json.dump(json_data, outfile, indent=2)

In [17]:
add_file_parts('workflow_soykb_decorated_decorated.json', {'genotype_gvcfs':0.05, 'haplotype_caller':0.05})